In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from tqdm.notebook import tqdm

## Task 1

In [ ]:
t, x = smp.symbols("t x", real=True)
D, M = smp.symbols("D M", real=True)
u = smp.symbols("u", cls=smp.Function)
u = u(x, t)
# Diffusion equation
diffEq = smp.Eq(smp.diff(u, t), D * smp.diff(u, x, 2))
# Gaussian function
gaussianU = (M / smp.sqrt(4 * smp.pi * D * t)) * (smp.exp((-x*x) / (4 * D * t)))

In [ ]:
diffEq

In [ ]:
gaussianU

In [ ]:
# Checking if gaussian is indeed a solution to diffEq
smp.checkpdesol(diffEq, gaussianU)


## IT IS!!

In [ ]:
# Creating the numerical function
gaussianU_f = smp.lambdify([x, t, D, M], gaussianU, "numpy")

In [ ]:
D_val = 10
M_val = 10

In [ ]:
xValues = np.arange(-100, 100, dtype=float)
for t_val in [_ * 5 for _ in range(1,6)]:
    yValues = gaussianU_f(xValues, t_val, D_val, M_val)
    plt.plot(xValues, yValues, label=f"t = {t_val}")
plt.legend()
plt.xlabel("X")
plt.ylabel("u(x,t)")

## Task 2

In [ ]:
def uNK_initial(x: float, L: float) -> float:
    '''
    Returns the initial condition of the field at position x

        Parameters:
            x (float): position of the field
            L (float): Size of the system
        
        Returns:
            initialCondition (float): Initial condition at point x
    '''
    initialCondition = np.power(4 * x * (L - x) / np.square(L), 10)
    return initialCondition

In [ ]:
def numSchemeStep(uArr: np.ndarray, n: float, deltaX: float, deltaT: float, D: float, K: int, boundaryCondition: bool):
        '''
        Calculates the next time step for all x
                
                Parameters:
                        uArr (numpy.ndarray): Array for calculated position(axis=0) and time(axis=1) steps
                        n (float): Current time step
                        deltaX (float): Size of the step in spatial dimension
                        deltaT (float): Size of the step in temporal dimension
                        D (float): The diffusion constant
                        K (int): Number of spatial steps
                        boundaryCondition (bool): Boundary condition of the system
                                0 or False for absorbing boundary conditions\n
                                1 or True for no-flux boundary conditions
                
                Returns:
                        nextStep (numpy.ndarray): Array for next time step calculated,
                                to be added as a column to the original uArr

        '''
        # Create an array for the next step
        nextStep = np.empty(uArr.shape[0])

        # Add an empty column if the array is 1-dim
        # Allows the double indices below to work smoothly
        if uArr.ndim == 1:
                uArr = np.c_[np.arange(uArr.shape[0]), uArr]
                # Also increase n as the number of columns have increased
                n = n + 1
                
        # Loop over all x
        for k in range(uArr.shape[0]):
                # Find the terms

                # Apply the adjustments for the boundaries accordingly
                if k == 0:
                        uNKMinus1 = uArr[1, n] if boundaryCondition else 0
                else:
                        uNKMinus1 = uArr[k - 1, n]
                if k == K:
                        uNKPlus1 = uArr[k - 1, n] if boundaryCondition else 0
                else:
                        uNKPlus1 = uArr[k + 1, n]
                uNK = uArr[k , n]

                # Calculate the next time step for the position k
                uNPlus1K = uNK + (deltaT * D * (uNKMinus1 - 2 * uNK + uNKPlus1) / deltaX**2)

                # Save it to the array
                nextStep[k] = uNPlus1K

        return nextStep

In [ ]:
def numScheme(initialConditionFunc, D: float, L: float, K: int, tMax: float, deltaFactor: float, boundaryCondition: bool):
    '''
    Calculates the numerical description of the diffusion equation

        Parameters:
            initialConditionFunc (function): Function that describes the initial condition for a given x in the form of func(x, L)
            D (float): Diffusion constant
            L (float): Size of the system
            K (int): Number of spatial steps
            tMax (float): Desired amount of time
            deltaFactor (float): Factor for determining deltaT
            boundaryCondition (bool): Boundary condition of the system
                    0 or False for absorbing boundary conditions\n
                    1 or True for no-flux boundary conditions
        
        Returns:
            uArr (numpy.ndarray): Description of the system both spatially(rows) and temporal(columns)
    '''
    # Calculating deltaX and deltaT
    deltaX = L / K
    deltaT = (deltaX ** 2 / (2 * D_val)) / deltaFactor
    # The number of time steps
    N = tMax / deltaT
    print(f"deltaX = {deltaX}, deltaT = {deltaT}, N= {N}\n")
    
    # Creating the ndarray
    uArr = np.empty((K + 1, int(N) + 1))
    uArr[:, 0] = np.arange(0, L + deltaX, deltaX)

    # Calculating the initial condition
    uArr[:, 0] = initialConditionFunc(uArr[:, 0], L)

    # Calculating all time steps
    for n in tqdm(range(int(N))):
        uArr[:, n + 1] = numSchemeStep(uArr, n, deltaX, deltaT, D, K, boundaryCondition)
    
    return uArr

    


In [ ]:
D_val = 1
L_val = 5
K_val = 500
tMax_val = 1
deltaFactor_val = 1
# Lasts around 15 minutes
noFlux = numScheme(uNK_initial, D_val, L_val, K_val, tMax_val, deltaFactor_val, True)

In [ ]:
for i in range(11):
    plt.plot(noFlux[:, i * 2000], label=f"t: {i*1/10}")
plt.legend()
plt.show()

In [ ]:
for i in range(5):
    idx = int(0.25/ 5e-5) * i
    plt.plot(noFlux[:, idx], label=f"t = {0.25*i}")
plt.legend()
plt.show()

In [ ]:
for i in range(11):
    print(f"total u at t = {i / 10} =", np.sum(noFlux[:, i * 200000]))
    if i == 10:
        percentChange = ( ( np.sum(noFlux[:, 0]) - np.sum(noFlux[:, 2000000])) / np.sum(noFlux[:, 0]) ) * 100
        print(f"{percentChange} % percent change" )

In [ ]:
D_val = 1
L_val = 5
K_val = 500
tMax_val = 20
deltaFactor_val = 1
# Lasts around 16 minutes
absorbing = numScheme(uNK_initial, D_val, L_val, K_val, tMax_val, deltaFactor_val, False)

In [ ]:
for i in range(5):
    idx = int(0.25/ 5e-7) * i
    plt.plot(absorbing[:, idx], label=f"t = {0.25*i}")
plt.legend()
plt.show()

In [ ]:
for i in range(21):
    plt.plot(absorbing[:, i * 20000], label=f"t: {i*1}")
plt.legend()
plt.show()

In [ ]:
for i in range(21):
    print(f"total u at t = {i} =", np.sum(absorbing[:, i * 20000]))
    if i == 20:
        percentChange = ( ( np.sum(absorbing[:, 0]) - np.sum(absorbing[:, 400000])) / np.sum(absorbing[:, 0]) ) * 100
        print(f"{percentChange} % percent change" )

In [ ]:
amount = np.empty(400001)
amount.shape

In [ ]:
for i in tqdm(range(absorbing.shape[1])):
    amount[i] = np.sum(absorbing[:, i])

In [ ]:
plt.plot(amount)
plt.xticks([      0.,  50000., 100000., 150000., 200000., 250000.,
        300000., 350000., 400000.], [ 0., 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20])

In [ ]:
plt.semilogy(amount)
plt.xticks([      0.,  50000., 100000., 150000., 200000., 250000.,
        300000., 350000., 400000.], [ 0., 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20])

In [ ]:
slope, intercept = np.polyfit(np.log(amount), np.arange(amount.shape[0]), 1)
print(f"slope: {slope/20000}, intercept: {intercept/20000}")

In [ ]:
plt.plot(np.arange(amount.shape[0]), np.log(amount))